## Reccurent Neural Network: Pytorch Implementation

In [19]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(['science', 'notebook', 'grid'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
# Generating Synthetic Data

def generate_time_series(n_points=10000):
    time = np.linspace(0, 10, n_points)
    signal = np.sin(time) 
    return time, signal

time, signal = generate_time_series()

# Splitting  data into train and test sets
train_size = int(0.8 * len(time))
train_time, test_time = time[:train_size], time[train_size:]
train_signal, test_signal = signal[:train_size], signal[train_size:]

### Defining Models in Pytorch

While defining a RNN object we should take care of two essential parameters:

`input_size` — The number of expected features in the input x

`hidden_size` — The number of features in the hidden state h

The input_size is 1 since we are using one time step of each sequence. Also, we can use `n_layers` parameter to get a stacked RNN with n hidden layers.

Note: if `batch_first` = True, then batch dimension in input and output comes first.

#### Inputs

1. input shape (batch, seq_len, input_size): tensor containing the features of the input sequence.

2. h_0 of shape (batch, num_layers * num_directions,  hidden_size): tensor containing the initial hidden state for each element in the batch.

#### Outputs

1. output of shape (seq_len, batch, num_directions * hidden_size): tensor containing the output features (h_t) from the last layer of the RNN, for each t.

2. h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len.

In [21]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

In [22]:
train_data = torch.tensor(train_signal, dtype=torch.float32)
test_data = torch.tensor(test_signal, dtype=torch.float32)
signal_data = torch.tensor(signal, dtype=torch.float32)

# hyperparameters and model instances
input_size = 1
hidden_size = 64
output_size = 1
n_epochs = 100
learning_rate = 0.001

rnn_model = RNNModel(input_size, hidden_size, output_size)
lstm_model = LSTMModel(input_size, hidden_size, output_size)
gru_model = GRUModel(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
rnn_optimizer = torch.optim.Adam(rnn_model.parameters(), lr=learning_rate)
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=learning_rate)
gru_optimizer = torch.optim.Adam(gru_model.parameters(), lr=learning_rate)

In [23]:
# Training loop
for epoch in range(n_epochs):
    rnn_optimizer.zero_grad()
    lstm_optimizer.zero_grad()
    gru_optimizer.zero_grad()

    rnn_output = rnn_model(train_data.unsqueeze(0).unsqueeze(2))
    lstm_output = lstm_model(train_data.unsqueeze(0).unsqueeze(2))
    gru_output = gru_model(train_data.unsqueeze(0).unsqueeze(2))

    rnn_target = train_data[-1].view(1, 1)
    lstm_target = train_data[-1].view(1, 1)
    gru_target = train_data[-1].view(1, 1)

    rnn_loss = criterion(rnn_output, rnn_target)
    lstm_loss = criterion(lstm_output, lstm_target)
    gru_loss = criterion(gru_output, gru_target)

    rnn_loss.backward()
    lstm_loss.backward()
    gru_loss.backward()

    rnn_optimizer.step()
    lstm_optimizer.step()
    gru_optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], RNN Loss: {rnn_loss.item():.4f}, LSTM Loss: {lstm_loss.item():.4f}, GRU Loss: {gru_loss.item():.4f}')


Epoch [10/100], RNN Loss: 0.0001, LSTM Loss: 0.5074, GRU Loss: 0.2691
Epoch [20/100], RNN Loss: 0.0018, LSTM Loss: 0.0069, GRU Loss: 0.0063
Epoch [30/100], RNN Loss: 0.0051, LSTM Loss: 0.0050, GRU Loss: 0.0001
Epoch [40/100], RNN Loss: 0.0042, LSTM Loss: 0.0047, GRU Loss: 0.0070
Epoch [50/100], RNN Loss: 0.0019, LSTM Loss: 0.0039, GRU Loss: 0.0004
Epoch [60/100], RNN Loss: 0.0005, LSTM Loss: 0.0010, GRU Loss: 0.0003
Epoch [70/100], RNN Loss: 0.0001, LSTM Loss: 0.0000, GRU Loss: 0.0003
Epoch [80/100], RNN Loss: 0.0000, LSTM Loss: 0.0000, GRU Loss: 0.0000
Epoch [90/100], RNN Loss: 0.0000, LSTM Loss: 0.0000, GRU Loss: 0.0000
Epoch [100/100], RNN Loss: 0.0000, LSTM Loss: 0.0000, GRU Loss: 0.0000


In [24]:
# Evaluation
with torch.no_grad():
    rnn_model.eval()
    lstm_model.eval()
    gru_model.eval()

    rnn_test_output = rnn_model(test_data.unsqueeze(0).unsqueeze(2))
    lstm_test_output = lstm_model(test_data.unsqueeze(0).unsqueeze(2))
    gru_test_output = gru_model(test_data.unsqueeze(0).unsqueeze(2))

print(f'RNN Prediction: {rnn_test_output.item()}')
print(f'LSTM Prediction: {lstm_test_output.item()}')
print(f'GRU Prediction: {gru_test_output.item()}')

RNN Prediction: 0.5249654650688171
LSTM Prediction: 0.5862373113632202
GRU Prediction: 0.3224380910396576


In [25]:
# time series
plt.figure(figsize=(12, 6))
plt.plot(time, signal, label='Original Time Series', color='blue')

# Predictions made by RNN, LSTM, and GRU models at their respective time points
with torch.no_grad():
    rnn_preds = [rnn_model(signal_data[:i+1].unsqueeze(0).unsqueeze(2)).item() for i in range(len(time))]
    lstm_preds = [lstm_model(signal_data[:i+1].unsqueeze(0).unsqueeze(2)).item() for i in range(len(time))]
    gru_preds = [gru_model(signal_data[:i+1].unsqueeze(0).unsqueeze(2)).item() for i in range(len(time))]

plt.scatter(time, rnn_preds, label='RNN Prediction', marker='o', color='red')
plt.scatter(time, lstm_preds, label='LSTM Prediction', marker='x', color='green')
plt.scatter(time, gru_preds, label='GRU Prediction', marker='^', color='purple')

plt.xlabel('Time')
plt.ylabel('Signal')
plt.legend()
plt.title('Original Time Series and Model Predictions')
plt.savefig('without_random_noise.png')
plt.show()